<a href="https://colab.research.google.com/github/ghinanurazizah/IncomeifyApp-CH2-PS161/blob/ML/TensorFlow_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# Import library
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
df = pd.read_csv('/content/drive/MyDrive/Capstone Project/data.csv', sep='|')
df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary
0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,NaN
1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,NaN
2,3,SALES ADMIN,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",30 days,51 - 200 pekerja,Umum & Grosir,Internal Sales & AdminJob Description :We are ...,NaN
3,4,City Operation Lead Shopee Express (Cirebon),Cirebon,IDR,Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Pelayanan,Logistik/Rantai Pasokan","Tip;Waktu regular, Senin - Jumat;Kasual (conto...",21 days,2001 - 5000 pekerja,Retail/Merchandise,Job Description:Responsible for HSE implementa...,NaN
4,5,Japanese Interpreter,Bekasi,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Lainnya,Jurnalis/Editor",NaN,23 days,201 - 500 pekerja,Manajemen/Konsulting HR,Overview: Our clients is manufacture for autom...,NaN


Data Preprocessing

In [30]:
# Delete USD currency
df = df[df['salary_currency'] != 'USD']
# Drop unnecesary colomns
df.drop(['id', 'salary_currency', 'job_title', 'job_function', 'job_benefits', 'company_process_time', 'company_size', 'company_industry', 'job_description'], axis=1, inplace=True)
# Drop null
df = df.dropna()
# Drop duplicate values
df.drop_duplicates()
df.head()

,location,career_level,experience_level,education_level,employment_type,salary
9,Banten,Supervisor/Koordinator,3 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,8000000.0
18,Jakarta Pusat,Pegawai (non-manajemen & non-supervisor),1 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,4750000.0
22,Surabaya,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,5250000.0
26,Jakarta Pusat,Manajer/Asisten Manajer,2 tahun,Tidak terspesifikasi,Penuh Waktu,15000000.0
28,Papua,Pegawai (non-manajemen & non-supervisor),1 tahun,Tidak terspesifikasi,Penuh Waktu,5040000.0


In [31]:
# Remove 'tahun' in experience_level
pattern = re.compile('[a-zA-Z]')
experience = [re.sub(pattern, '', s) for s in df['experience_level']]
df['experience_level'] = experience
df['experience_level'] = df['experience_level'].astype(int)
df.head()

,location,career_level,experience_level,education_level,employment_type,salary
9,Banten,Supervisor/Koordinator,3,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,8000000.0
18,Jakarta Pusat,Pegawai (non-manajemen & non-supervisor),1,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,4750000.0
22,Surabaya,Pegawai (non-manajemen & non-supervisor),2,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,5250000.0
26,Jakarta Pusat,Manajer/Asisten Manajer,2,Tidak terspesifikasi,Penuh Waktu,15000000.0
28,Papua,Pegawai (non-manajemen & non-supervisor),1,Tidak terspesifikasi,Penuh Waktu,5040000.0


In [32]:
# Remove employment_type that has 2 options
df = df[(df['employment_type'] != 'Penuh Waktu, Kontrak') & (df['employment_type'] != 'Penuh Waktu, Paruh Waktu') & (df['employment_type'] != 'Kontrak, Temporer')]
# Remove unspecified education_level
df = df[(df['education_level'] != 'Tidak terspesifikasi')]

df.head()

,location,career_level,experience_level,education_level,employment_type,salary
9,Banten,Supervisor/Koordinator,3,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,8000000.0
18,Jakarta Pusat,Pegawai (non-manajemen & non-supervisor),1,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,4750000.0
22,Surabaya,Pegawai (non-manajemen & non-supervisor),2,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,5250000.0
31,Tangerang,Manajer/Asisten Manajer,5,Sarjana (S1),Penuh Waktu,6600000.0
34,Jakarta Timur,Supervisor/Koordinator,2,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,9500000.0


In [33]:
df['education_level'] = df['education_level'].str.split(',').str[-1].str.strip()
df

,location,career_level,experience_level,education_level,employment_type,salary
9,Banten,Supervisor/Koordinator,3,Sarjana (S1),Penuh Waktu,8000000.0
18,Jakarta Pusat,Pegawai (non-manajemen & non-supervisor),1,Sarjana (S1),Penuh Waktu,4750000.0
22,Surabaya,Pegawai (non-manajemen & non-supervisor),2,Sarjana (S1),Penuh Waktu,5250000.0
31,Tangerang,Manajer/Asisten Manajer,5,Sarjana (S1),Penuh Waktu,6600000.0
34,Jakarta Timur,Supervisor/Koordinator,2,Sarjana (S1),Penuh Waktu,9500000.0
...,...,...,...,...,...,...
34741,Bekasi,Pegawai (non-manajemen & non-supervisor),2,Sarjana (S1),Kontrak,4500000.0
34742,Jakarta Pusat,Pegawai (non-manajemen & non-supervisor),2,Sarjana (S1),Penuh Waktu,11500000.0
34743,Jakarta Selatan,Pegawai (non-manajemen & non-supervisor),2,Sarjana (S1),Penuh Waktu,6250000.0
34744,Palembang,Lulusan baru/Pengalaman kerja kurang dari 1 tahun,1,SMU/SMK/STM,Penuh Waktu,3387500.0


In [34]:
df['career_level'].unique()

array(['Supervisor/Koordinator',
       'Pegawai (non-manajemen & non-supervisor)',
       'Manajer/Asisten Manajer',
       'Lulusan baru/Pengalaman kerja kurang dari 1 tahun',
       'CEO/GM/Direktur/Manajer Senior'], dtype=object)

In [35]:
# Remove outlier
Q1 = df['experience_level'].quantile(0.25)
Q3 = df['experience_level'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df[(df['experience_level'] > lower_bound) & (df['experience_level'] < upper_bound)]
df

,location,career_level,experience_level,education_level,employment_type,salary
9,Banten,Supervisor/Koordinator,3,Sarjana (S1),Penuh Waktu,8000000.0
18,Jakarta Pusat,Pegawai (non-manajemen & non-supervisor),1,Sarjana (S1),Penuh Waktu,4750000.0
22,Surabaya,Pegawai (non-manajemen & non-supervisor),2,Sarjana (S1),Penuh Waktu,5250000.0
31,Tangerang,Manajer/Asisten Manajer,5,Sarjana (S1),Penuh Waktu,6600000.0
34,Jakarta Timur,Supervisor/Koordinator,2,Sarjana (S1),Penuh Waktu,9500000.0
...,...,...,...,...,...,...
34741,Bekasi,Pegawai (non-manajemen & non-supervisor),2,Sarjana (S1),Kontrak,4500000.0
34742,Jakarta Pusat,Pegawai (non-manajemen & non-supervisor),2,Sarjana (S1),Penuh Waktu,11500000.0
34743,Jakarta Selatan,Pegawai (non-manajemen & non-supervisor),2,Sarjana (S1),Penuh Waktu,6250000.0
34744,Palembang,Lulusan baru/Pengalaman kerja kurang dari 1 tahun,1,SMU/SMK/STM,Penuh Waktu,3387500.0


In [36]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Fit and transform the 'NewColumnName' column
df['education_level'] = label_encoder.fit_transform(df['education_level'])
df['career_level'] = label_encoder.fit_transform(df['career_level'])
df['location'] = label_encoder.fit_transform(df['location'])
df['employment_type'] = label_encoder.fit_transform(df['employment_type'])
df['experience_level'] = label_encoder.fit_transform(df['experience_level'])
# Print the resulting DataFrame
df

<ipython-input-36-642a67c21e1b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['education_level'] = label_encoder.fit_transform(df['education_level'])
<ipython-input-36-642a67c21e1b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['career_level'] = label_encoder.fit_transform(df['career_level'])
<ipython-input-36-642a67c21e1b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,location,career_level,experience_level,education_level,employment_type,salary
9,13,4,2,4,2,8000000.0
18,47,3,0,4,2,4750000.0
22,148,3,1,4,2,5250000.0
31,151,2,4,4,2,6600000.0
34,50,4,1,4,2,9500000.0
...,...,...,...,...,...,...
34741,18,3,1,4,0,4500000.0
34742,47,3,1,4,2,11500000.0
34743,49,3,1,4,2,6250000.0
34744,107,1,0,3,2,3387500.0


In [37]:
df.shape

(6859, 6)

In [38]:
# Normalized salary
df['salary'] = df['salary'] / 10000000
df

<ipython-input-38-508bff2014e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['salary'] = df['salary'] / 10000000


,location,career_level,experience_level,education_level,employment_type,salary
9,13,4,2,4,2,0.80000
18,47,3,0,4,2,0.47500
22,148,3,1,4,2,0.52500
31,151,2,4,4,2,0.66000
34,50,4,1,4,2,0.95000
...,...,...,...,...,...,...
34741,18,3,1,4,0,0.45000
34742,47,3,1,4,2,1.15000
34743,49,3,1,4,2,0.62500
34744,107,1,0,3,2,0.33875


In [39]:
# Split feature (X) and target (y)
X = df[['career_level', 'location', 'experience_level', 'education_level', 'employment_type']]
y = df['salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Build Model

In [55]:
from tensorflow.keras.losses import Huber

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, input_shape=(X_train.shape[1],)),
  tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(l1=0.01)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(l1=0.01)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(l1=0.01)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation='linear')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)

model.compile(optimizer=optimizer, loss=Huber(), metrics=['mae'])

model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=300)

Epoch 1/300
172/172 [==============================] - 3s 7ms/step - loss: 11.6152 - mae: 1.9426 - val_loss: 10.2209 - val_mae: 0.6687
Epoch 2/300
172/172 [==============================] - 1s 5ms/step - loss: 10.1060 - mae: 0.8559 - val_loss: 9.6648 - val_mae: 0.6134
Epoch 3/300
172/172 [==============================] - 1s 5ms/step - loss: 9.4741 - mae: 0.6921 - val_loss: 9.1801 - val_mae: 0.5989
Epoch 4/300
172/172 [==============================] - 1s 5ms/step - loss: 8.9315 - mae: 0.6212 - val_loss: 8.7065 - val_mae: 0.5915
Epoch 5/300
172/172 [==============================] - 1s 5ms/step - loss: 8.4207 - mae: 0.5660 - val_loss: 8.2422 - val_mae: 0.5787
Epoch 6/300
172/172 [==============================] - 1s 3ms/step - loss: 7.9375 - mae: 0.5195 - val_loss: 7.7883 - val_mae: 0.5582
Epoch 7/300
172/172 [==============================] - 1s 3ms/step - loss: 7.4830 - mae: 0.4859 - val_loss: 7.3475 - val_mae: 0.5212
Epoch 8/300
172/172 [==============================] - 1s 3ms/step

In [56]:
# Trying prediction
model.predict([[4, 13, 3, 4, 2]])

1/1 [==============================] - 0s 103ms/step


array([[0.76783395]], dtype=float32)

In [59]:
# Saving model
model.save("SalaryModel.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
